# Using the lcvr_learning Class to Measure Training Data

To train our model, it helps to have a large amount of training data. While this can be measured and calibrated by hand, it can take several hours for a single wavelength to get the data. On top of this, changing the orientation of the LCVR's will then change the necessary driving voltage for different polarizations, resulting in prior training data being useless. To combat this, the class has an automated function for collecting training data.

In [1]:
import lcvr_learning as lcl
import numpy as np
import pyvisa
import pandas as pd
rm = pyvisa.ResourceManager('@py')
lcvrs = lcl.lcvr_learning(0x6a,0x6b)

In [4]:
lcvrs.set_input_volts(4,1)

In [11]:
data = lcvrs.get_training_data(200,480)

Starting training data scan. Don't touch anything please


In [12]:
data.head()

,Wavelength,V1,V2,Out
0,480,1.000000,1.0,-0.335938
1,480,1.061224,1.0,-0.336234
2,480,1.122449,1.0,-0.336219
3,480,1.183673,1.0,-0.336219
4,480,1.244898,1.0,-0.336250


In [10]:
lcvrs.get_voltage()

-0.13998437500000005

In [13]:
data.to_csv('trainingdata_new/480.csv')